<a href="https://colab.research.google.com/github/LSDtopotools/lsdtt_notebooks/blob/master/lsdtopotools/basic_examples/swath_plot_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Plotting a swath profile

Last updated by Simon M Mudd 08/05/2023

`lsdtt-basic-metrics` has a swath plotting routine, we will use that here to plot a swath profile. 

## Stuff we need to do if you are in colab (not required in the lsdtopotools pytools container)

**If you are in the `docker_lsdtt_pytools` docker container, you do not need to do any of this. 
The following is for executing this code in the google colab environment only.**

If you are in the docker container you can skip to the **First get data** section. 

First we install `lsdtopotools`. The first line downloads the package and the second installs it. The `/dev/null` stuff is just to stop the notebook printing a bunch of text to screen.  

In [ ]:
!wget https://pkgs.geos.ed.ac.uk/geos-jammy/pool/world/l/lsdtopotools2/lsdtopotools2_0.9-1geos~22.04.1_amd64.deb  &> /dev/null
!apt install ./lsdtopotools2_0.9-1geos~22.04.1_amd64.deb  &> /dev/null

The next line tests to see if it worked. If you get some output asking for a parameter file then `lsdtopotools` is installed. This notebook was tested on version 0.9.

In [ ]:
!lsdtt-basic-metrics -v

Now we install `lsdviztools`:

In [ ]:
!pip install lsdviztools  &> /dev/null

## First get data

We need to get some data to download. 

We are going to get some data from the centre of Lesotho, in some small catchements draining to the Orange River. 

We are going to download data using the opentopography scraper that is included with `lsdviztools`. You will need to get an opentopography.org account and copy in your API key. 

You can sign up to an opentopography.org account here: https://portal.opentopography.org/myopentopo 

In [ ]:
import lsdviztools.lsdbasemaptools as bmt
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio

# YOU NEED TO PUT YOUR API KEY IN A FILE
your_OT_api_key_file = "my_OT_api_key.txt"

with open(your_OT_api_key_file, 'r') as file:
    print("I am reading you OT API key from the file "+your_OT_api_key_file)
    api_key = file.read().rstrip()
    print("Your api key starts with: "+api_key[0:4])

Dataset_prefix = "Lesotho"
source_name = "COP30"

Xian_DEM = bmt.ot_scraper(source = source_name,
                        lower_left_coordinates = [-29.986795303183285, 28.210294055430822], 
                        upper_right_coordinates = [-29.546820300795922, 28.636351905601636],
                        prefix = Dataset_prefix, 
                        api_key_file = your_OT_api_key_file)
Xian_DEM.print_parameters()
Xian_DEM.download_pythonic()

In [ ]:
DataDirectory = "./"
Fname = Dataset_prefix+"_"+source_name+".tif"
gio.convert4lsdtt(DataDirectory,Fname)

Let's check to see what the filenames we generated are:

In [ ]:
!ls Lesotho* -l

We have a little script to plot the basemap. The `-bmrem` flag controls how wide the map is compared to the DEM time.

In [ ]:
!lsdtt_plotbasicrasters -dir ./ -fname Lesotho_COP30_UTM -BM true -bmrem 20

The data goes into a directory called `basemap_plots`. We can see the image by using the `display` function.

In [ ]:
!ls basemap_plots

In [ ]:
from IPython.display import display, Image
display(Image(filename="./basemap_plots/Lesotho_COP30_UTM_basemap.png", width=800))

## Now extract swath between two points

We can extract a swath between two points. We will need to make the start and end point in a pandas dataframe, and save to `csv`. I get the start and end points by right clicking in google maps and copying the lat-long. 

**IMPORTANT** If you want to extract a channel you need the upstream point first.

In [ ]:
# Import pandas library
import pandas as pd

data = [ [-29.621021116107862, 28.57147034542054],
         [-29.783077317434813, 28.281779758189604]
       ]

# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['latitude', 'longitude'])

df.to_csv("swath1.csv",index=False)
df.head()

## Now run the swath tool. 

In [ ]:
## Get the basins and the channel profile
import lsdviztools.lsdmapwrappers as lsdmw

Dataset_prefix = "Lesotho"
source_name = "COP30"

lsdtt_parameters = {"write_hillshade" : "true",
                    "calculate_swath_profile" : "true",
                    "calculate_swath_along_line" : "true",
                    "swath_points_csv" : "swath1.csv"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-basic-metrics", 
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

We can print the footprint of the swath using `lsdviztools`:

In [ ]:
!ls Lesotho*

In [ ]:
### Plot the swath 
Base_file = "Lesotho_COP30_UTM"
Drape_prefix = Base_file+"_swathval"
DataDirectory = "./"
this_area_img = lsdmw.SimpleDrape(DataDirectory,Base_file,Drape_prefix, cmap="jet", 
                                       cbar_label = "Swath distance (m)",
                                       cbar_loc = "bottom",
                                       save_fig=True, size_format="ESURF",dpi=600)


Hey? Where is my image??? Because we used `save_fig=True`, the image is not displayed but rather printed to file. So we can look at it with a call to `display`:


In [ ]:
print(this_area_img)
from IPython.display import display, Image
display(Image(filename=this_area_img, width=600))

We can plot this swath with `lsdviztools`, with either the command line script:

In [ ]:
!ls *.csv

In [ ]:
!lsdtt_plotbasicrasters -dir ./ -fname Lesotho_COP30_UTM -PS True -swath_prefix Lesotho_COP30_UTM_swath

In [ ]:
!ls *.png

In [ ]:
from IPython.display import display, Image
display(Image(filename="test_swath.png", width=800))

Or you can access this directly from `lsdviztools`

In [ ]:
import lsdviztools.lsdplottingtools.lsdmap_swathplotting as lsdsp

In [ ]:
import lsdviztools.lsdmapwrappers as lsdmw
DataDirectory="./"
imname = lsdmw.SimpleSwath(DataDirectory,"Lesotho_COP30_UTM_swath.csv", fig_format = "png", dpi = 500, aspect_ratio = 2)

In [ ]:
from IPython.display import display, Image
display(Image(filename=imname, width=800))

## Get swath along channel

A straight line swath is a little silly. Also, the swath was kind of narrow. We can get the swath along the channel as well as have a wider swath by changing a few parameters. In this case we swich to `calculate_swath_along_channel` and set the swath width to 3 km with ```"swath_width" : "3000"```

In [ ]:
## Get the basins and the channel profile
import lsdviztools.lsdmapwrappers as lsdmw

Dataset_prefix = "Lesotho"
source_name = "COP30"

lsdtt_parameters = {"print_hillshade" : "true",
                    "calculate_swath_profile" : "true",
                    "calculate_swath_along_channel" : "true",
                    "swath_points_csv" : "swath1.csv",
                    "swath_width" : "3000"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-basic-metrics", 
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

Okay, lets have a look at the result. 

In [ ]:
### Plot the swath
Base_file = "Lesotho_COP30_UTM"
Drape_prefix = Base_file+"_swathval"
DataDirectory = "./"
this_area_img = lsdmw.SimpleDrape(DataDirectory,Base_file,Drape_prefix, cmap="jet", 
                                       cbar_label = "Swath distance (m)",
                                       cbar_loc = "bottom",
                                       save_fig=True, size_format="ESURF",dpi=600)

In [ ]:
print(this_area_img)
from IPython.display import display, Image
display(Image(filename=this_area_img, width=600))

Okay, that looks a bit better. Lets plot it. 

In [ ]:
lsdsp.PlotSwath("Lesotho_COP30_UTM_swath.csv", FigFileName = 'channel_swath.png', 
                fig_format = "png", dpi = 500, aspect_ratio = 2)

In [ ]:
from IPython.display import display, Image
display(Image(filename="channel_swath.png", width=600))

You will see in this case that the bottom of the swath elevation is the channel profile. 